In [1]:
######################################     Modules     #######################################
# MODULES
import sys
from time import process_time
import pickle
import pandas as pd

sys.path.append('../.')
import verbose_module
sys.path.append('../../../.')
import pIRPgym


computer_name = input("Running experiment on mac? [Y/n]")
if computer_name == '': 
    path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPgym/'
    experiments_path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Column Generation/'
else: 
    path = 'C:/Users/jm.betancourt/Documents/Research/pIRPgym/'
    experiments_path = 'G:/Mi unidad/Research/Supply Chain Analytics/Experiments/Column Generation/'

verbose = True
alphas = [0.1,0.2,0.4,0.6]

Experiments = [f'Experiment {i}' for i in range(1,6)]    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


# Scalability

In [8]:
Experiments_results = {'M/K':[5,10,15,20,40],'CG':[]}
Experiments_results.update({alpha:[] for alpha in alphas})

for experiment in Experiments:

    reliability = {alpha:[] for alpha in alphas}
    reliability.update({'CG':[]})
    for replica in range(1,4):
        # Upload results
        with open(experiments_path+f'{experiment}/Replica {replica}/instance_information.pkl', 'rb') as file:
            inst_info = pickle.load(file)

        with open(experiments_path+f'{experiment}/Replica {replica}/CG.pkl', 'rb') as file:
            CG_results = pickle.load(file)
        CG_alpha_results = dict()
        for alpha in alphas: 
            with open(experiments_path+f'{experiment}/Replica {replica}/CG_{alpha}.pkl', 'rb') as file:
                CG_alpha_results[alpha] = pickle.load(file)
        
        # For each day determine the best score
        for t in range(len(CG_results)):
            best_performance = CG_results[t][1]
            for alpha in alphas: 
                if CG_alpha_results[alpha][t][1] < best_performance:
                    best_performance = CG_alpha_results[alpha][t][1]
        
            reliability['CG'].append(int(CG_results[t][1] == best_performance))
            for alpha in alphas: 
                reliability[alpha].append(int(CG_alpha_results[alpha][t][1] == best_performance))

    # Storing the results
    Experiments_results['CG'].append(round(sum(reliability["CG"])/len(reliability["CG"]),4))
    for alpha in alphas:
        Experiments_results[alpha].append(round(sum(reliability[alpha])/len(reliability[alpha]),4))


df = pd.DataFrame(Experiments_results)
df.index = Experiments
df.columns = ['M/K','Regular', 'al 0.1','al 0.2','al 0.4','al 0.6']
df = df.style.format(dict.fromkeys(list(df.columns)[1:],"{:.2%}"))
df

,M/K,Regular,al 0.1,al 0.2,al 0.4,al 0.6
Experiment 1,5,50.00%,91.67%,91.67%,97.22%,97.22%
Experiment 2,10,61.11%,63.89%,63.89%,77.78%,91.67%
Experiment 3,15,13.89%,27.78%,36.11%,63.89%,86.11%
Experiment 4,20,36.11%,33.33%,55.56%,69.44%,69.44%
Experiment 5,40,13.89%,16.67%,30.56%,25.00%,36.11%


In [17]:
import pandas as pd

# Sample data
data = {
    ('Experiment1', 'Alpha_0.1'): [10, 20, 30, 40, 50, 60],
    ('Experiment1', 'Alpha_0.2'): [70, 80, 90, 100, 110, 120],
    ('Experiment1', 'Alpha_0.3'): [130, 140, 150, 160, 170, 180],
    ('Experiment2', 'Alpha_0.1'): [11, 21, 31, 41, 51, 61],
    ('Experiment2', 'Alpha_0.2'): [71, 81, 91, 101, 111, 121],
    ('Experiment2', 'Alpha_0.3'): [131, 141, 151, 161, 171, 181],
}

# Create the DataFrame with the multi-level columns
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


  Experiment1                     Experiment2                    
    Alpha_0.1 Alpha_0.2 Alpha_0.3   Alpha_0.1 Alpha_0.2 Alpha_0.3
0          10        70       130          11        71       131
1          20        80       140          21        81       141
2          30        90       150          31        91       151
3          40       100       160          41       101       161
4          50       110       170          51       111       171
5          60       120       180          61       121       181


In [13]:
import pandas as pd

# Initialize a DataFrame to store the results
columns = ['Experiment', 'Simple CG'] + [f'CG {alpha}' for alpha in alphas]
results_df = pd.DataFrame(columns=columns)

for experiment in Experiments:
    reliability = {'Experiment': experiment, 'Simple CG': 0}
    for alpha in alphas:
        reliability[f'CG {alpha}'] = 0

    for replica in range(1, 4):
        with open(experiments_path + f'{experiment}/Replica {replica}/instance_information.pkl', 'rb') as file:
            inst_info = pickle.load(file)

        with open(experiments_path + f'{experiment}/Replica {replica}/CG.pkl', 'rb') as file:
            CG_results = pickle.load(file)
        CG_alpha_results = {}
        for alpha in alphas:
            with open(experiments_path + f'{experiment}/Replica {replica}/CG_{alpha}.pkl', 'rb') as file:
                CG_alpha_results[alpha] = pickle.load(file)

        for t in range(len(CG_results)):
            best_performance = CG_results[t][1]
            for alpha in alphas:
                if CG_alpha_results[alpha][t][1] < best_performance:
                    best_performance = CG_alpha_results[alpha][t][1]

            reliability['Simple CG'] += int(CG_results[t][1] == best_performance)
            for alpha in alphas:
                reliability[f'CG {alpha}'] += int(CG_alpha_results[alpha][t][1] == best_performance)

    for key in reliability:
        reliability[key] = round(reliability[key] / (len(CG_results) * 3) * 100, 2)

    # Append the results to the DataFrame
    results_df = results_df.append(reliability, ignore_index=True)

# Display the DataFrame
print(results_df)


TypeError: unsupported operand type(s) for /: 'str' and 'int'

# Instance Visualization

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':12,'K':12,'T':8,'F':14,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 15       # Random seeds
det_rd_seed = 35

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    10	| 6.78	  57 	   3 	6007.6 	| 13.58	   3 	  52 	   3 	5999.6 	| 13.42	   6 	  38 	   3 	5999.6 	| 12.84	   85 	  13 	   3 	5999.6 	| 12.42	   287 	  10 	   3 	6000.1 	|
1    8	| 0.64	  24 	   3 	5852.4 	| 5.75	   5 	  26 	   3 	5852.3 	| 5.70	   7 	  26 	   3 	5852.3 	| 6.72	 

# M = 15, K = 15

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':15,'K':15,'T':7,'F':14,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 15       # Random seeds
det_rd_seed = 35

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    11	| 1615	  176 	   3 	6073.8 	| 76.62	  3	  50	   3 	6073.8 	| 47.94	  12	  40	   3 	6073.8 	| 49.25	  48	  40	   3 	6073.8 	| 98.22	  341	  25	   3 	6073.8 	|
1    8	| 0.67	  33 	   3 	5963.6 	| 5.27	  5	  10	   3 	5963.6 	| 5.50	  13	  12	   3 	5963.6 	| 5.70	  14	  12	   3 	

# M = 18, K = 18

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':18,'K':18,'T':7,'F':18,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 16       # Random seeds
det_rd_seed = 36

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    16	| 1870	  293 	   2 	4309.2 	| 441.4	  29	  107	   2 	4327.6 	| 843.9	  736	  112	   2 	4316.6 	| 901.8	  36241	  39	   2 	4309.2 	| 745.7	  49739	  66	   2 	4309.2 	|
1    17	| 1958	  313 	   2 	4361.9 	| 508.2	  62	  145	   2 	4236.6 	| 1959	  6352	  70	   2 	4203.6 	| 1842	

# M = 25, K = 25

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':25,'K':25,'T':5,'F':25,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 116       # Random seeds
det_rd_seed = 136

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    23	| 1803	  450 	   3 	5335.2 	| 1852	  2359	  147	   20 	41664.5	| 1351	  21021	  65	   3 	5278	| 1837	  43562	  117	   3 	5500.7	| 1818	  66394	  186	   21 	47526.0	|
1    16	| 73.12	  215 	   2 	4417.4 	| 74.02	  118	  116	   2 	4417.4	| 89.19	  3610	  56	   2 	4417.4	| 79.33